## Step 1: Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install dependencies
!pip install -q torch pandas numpy scikit-learn stable-baselines3 gymnasium python-dotenv

In [ ]:
# Mount Google Drive (for data and model storage)
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully")

## Step 2: Clone Repository from GitHub

In [ ]:
# Clone your repository (REPLACE WITH YOUR GITHUB URL)
!git clone https://github.com/YOUR_USERNAME/fluxpointai-backend.git

# Or if already cloned, pull latest changes
# %cd fluxpointai-backend
# !git pull origin main

In [ ]:
# Navigate to project directory
%cd fluxpointai-backend/fluxpoint

# Add to Python path
import sys
sys.path.insert(0, '/content/fluxpointai-backend/fluxpoint')

print("✅ Repository loaded")

## Step 3: Prepare Training Data

You need historical OHLCV data. Choose one option:
- **Option A**: Load from Google Drive (recommended)
- **Option B**: Download sample data
- **Option C**: Fetch from MT5 (requires MT5 setup)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# OPTION A: Load from Google Drive (if you already uploaded data)
# Make sure you have uploaded EURUSD_H1.csv to Drive
try:
    candles = pd.read_csv('/content/drive/MyDrive/fluxpoint/data/EURUSD_H1.csv')
    print(f"✅ Loaded {len(candles)} candles from Drive")
except FileNotFoundError:
    print("⚠️  Data not found in Drive. Using sample data generation...")
    
    # OPTION B: Generate sample data for testing
    # (Remove this in production - use real data!)
    dates = pd.date_range(start='2020-01-01', end='2024-01-01', freq='H')
    np.random.seed(42)
    
    # Simulate realistic price movements
    returns = np.random.normal(0, 0.0005, len(dates))
    prices = 1.0900 * np.exp(np.cumsum(returns))
    
    candles = pd.DataFrame({
        'timestamp': dates,
        'open': prices,
        'high': prices * (1 + np.abs(np.random.normal(0, 0.0005, len(dates)))),
        'low': prices * (1 - np.abs(np.random.normal(0, 0.0005, len(dates)))),
        'close': prices * (1 + np.random.normal(0, 0.0003, len(dates))),
        'volume': np.random.randint(500, 2000, len(dates))
    })
    
    print(f"✅ Generated {len(candles)} sample candles")
    print("⚠️  WARNING: Using simulated data. Upload real data to Drive for production!")

# Format data
candles['timestamp'] = pd.to_datetime(candles['timestamp'])
candles = candles.sort_values('timestamp').reset_index(drop=True)

# Display info
print(f"\nData Range: {candles['timestamp'].min()} to {candles['timestamp'].max()}")
print(f"Total Candles: {len(candles)}")
print(f"\nFirst few rows:")
print(candles.head())

## Step 4: Generate Training Data (TCE Validation)

Run TCE validation on historical data to create training examples.

In [ ]:
# Import TCE validation
from trading.tce.validation import validate_tce
from trading.tce.feature_engineering import extract_features

print("Generating training data from historical TCE setups...")
print("This may take several minutes...\n")

training_data = []
valid_setups = []

# Slide through history and validate TCE setups
for i in range(200, len(candles) - 100):  # Need lookback and forward data
    if i % 1000 == 0:
        print(f"Progress: {i}/{len(candles)} candles processed...")
    
    # Get candle window
    window = candles.iloc[i-200:i+100].copy()
    
    try:
        # Validate TCE setup at this point
        result = validate_tce(
            symbol='EURUSD',
            timeframe='H1',
            candles=window,
            current_idx=200,  # Current candle in window
            account_balance=10000,
            risk_percentage=1.0
        )
        
        if result.get('is_valid', False):
            # Extract features
            features = extract_features(window, current_idx=200)
            
            # Determine outcome (did TP or SL hit first?)
            entry_price = window.iloc[200]['close']
            sl_price = result['stop_loss']
            tp_price = result['take_profit']
            
            # Check future candles
            future = window.iloc[201:]
            tp_hit = (future['high'] >= tp_price).any()
            sl_hit = (future['low'] <= sl_price).any()
            
            if tp_hit or sl_hit:
                tp_idx = future[future['high'] >= tp_price].index[0] if tp_hit else 999999
                sl_idx = future[future['low'] <= sl_price].index[0] if sl_hit else 999999
                
                label = 1 if tp_idx < sl_idx else 0  # 1 = TP hit first, 0 = SL hit first
                
                training_data.append({
                    'features': features,
                    'label': label,
                    'timestamp_idx': i
                })
                
                # Also save for RL training
                result['features'] = features
                result['timestamp_idx'] = i
                valid_setups.append(result)
    
    except Exception as e:
        continue

print(f"\n✅ Generated {len(training_data)} training examples")
print(f"✅ Found {len(valid_setups)} valid TCE setups")

# Calculate stats
if len(training_data) > 0:
    wins = sum(1 for d in training_data if d['label'] == 1)
    win_rate = wins / len(training_data)
    print(f"\nHistorical Win Rate: {win_rate:.2%}")

## Step 5: Train ML Model (Deep Learning)

In [ ]:
# Prepare ML training data
import numpy as np

X = np.array([d['features'] for d in training_data])
y = np.array([d['label'] for d in training_data])

print(f"Training set shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Win rate in training set: {y.mean():.2%}")

In [ ]:
# Train ML model
from trading.tce.training import TCETrainer

print("Starting ML training...\n")

trainer = TCETrainer(model_name='tce_ml_model')

# Prepare data
train_loader, val_loader = trainer.prepare_data(
    X=X,
    y=y,
    validation_split=0.2,
    batch_size=32
)

# Train
metrics = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=100,
    learning_rate=0.001
)

print("\n" + "="*60)
print("ML TRAINING COMPLETE")
print("="*60)
print(f"Final Loss: {metrics['final_loss']:.4f}")
print(f"Validation Accuracy: {metrics['val_accuracy']:.2%}")
print(f"Training Time: {metrics['training_time']:.1f}s")
print("="*60)

## Step 6: Train RL Model (Execution Optimization)

In [ ]:
# Train RL model
from trading.rl.training import train_rl_execution

print("Starting RL training...\n")
print("This will train an agent to optimize trade execution.")
print("Expected time: 20-40 minutes\n")

rl_metrics = train_rl_execution(
    candles=candles,
    valid_setups=valid_setups,
    model_name='tce_execution_ppo',
    initial_balance=10000,
    total_timesteps=100000
)

print("\n" + "="*60)
print("RL TRAINING COMPLETE")
print("="*60)
print(f"Mean Reward: {rl_metrics['eval']['mean_reward']:.2f}")
print(f"Mean R-Multiple: {rl_metrics['eval']['mean_r_multiple']:.2f}R")
print(f"Win Rate: {rl_metrics['eval']['mean_win_rate']:.2%}")
print("="*60)

## Step 7: Save Models to Google Drive

In [ ]:
import os
from datetime import datetime

# Create directories in Drive
!mkdir -p /content/drive/MyDrive/fluxpoint/models/ml/
!mkdir -p /content/drive/MyDrive/fluxpoint/models/rl/
!mkdir -p /content/drive/MyDrive/fluxpoint/logs/

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Copy ML model
print("Saving ML model...")
!cp models/ml/tce_ml_model.pt /content/drive/MyDrive/fluxpoint/models/ml/tce_ml_model.pt
!cp models/ml/tce_ml_model.pt /content/drive/MyDrive/fluxpoint/models/ml/tce_ml_model_{timestamp}.pt

# Copy RL model
print("Saving RL model...")
!cp models/rl/tce_execution_ppo.zip /content/drive/MyDrive/fluxpoint/models/rl/tce_execution_ppo.zip
!cp models/rl/tce_execution_ppo.zip /content/drive/MyDrive/fluxpoint/models/rl/tce_execution_ppo_{timestamp}.zip

print("\n✅ Models saved to Google Drive")
print(f"   ML: /content/drive/MyDrive/fluxpoint/models/ml/tce_ml_model.pt")
print(f"   RL: /content/drive/MyDrive/fluxpoint/models/rl/tce_execution_ppo.zip")

In [ ]:
# Save training metrics
import json

log_data = {
    'timestamp': timestamp,
    'symbol': 'EURUSD',
    'timeframe': 'H1',
    'data_range': {
        'start': str(candles['timestamp'].min()),
        'end': str(candles['timestamp'].max()),
        'total_candles': len(candles)
    },
    'training_examples': len(training_data),
    'valid_setups': len(valid_setups),
    'ml_metrics': metrics,
    'rl_metrics': rl_metrics
}

log_path = f'/content/drive/MyDrive/fluxpoint/logs/training_{timestamp}.json'
with open(log_path, 'w') as f:
    json.dump(log_data, f, indent=2, default=str)

print(f"✅ Training log saved: {log_path}")

## Step 8: Test Trained Models

In [ ]:
# Load and test ML model
from trading.tce.ml_integration import TCEMLPredictor

predictor = TCEMLPredictor()
predictor.load_model('models/ml/tce_ml_model.pt')

# Test on a sample
test_features = X[0]
probability = predictor.predict(test_features)

print(f"Test Prediction:")
print(f"Features shape: {test_features.shape}")
print(f"Predicted probability: {probability:.2%}")
print(f"Actual outcome: {'Win' if y[0] == 1 else 'Loss'}")

## Training Complete! 🎉

### Next Steps:

1. **Download models from Google Drive** to your VPS:
   ```bash
   rclone sync gdrive:fluxpoint/models/ ~/fluxpointai-backend/fluxpoint/models/
   ```

2. **Restart Django** on VPS:
   ```bash
   sudo supervisorctl restart fluxpoint_django
   ```

3. **Test inference** on VPS:
   ```bash
   curl http://localhost:8000/api/tce/validate/
   ```

### Retraining Schedule:

- **Weekly**: Re-run this notebook with updated data
- **Monthly**: Full retrain with 2+ years of data
- **On-demand**: When strategy performance degrades

### Model Locations:

- ML Model: `/content/drive/MyDrive/fluxpoint/models/ml/tce_ml_model.pt`
- RL Model: `/content/drive/MyDrive/fluxpoint/models/rl/tce_execution_ppo.zip`
- Logs: `/content/drive/MyDrive/fluxpoint/logs/`